In [8]:
import requests
import config
import json
import os
import urllib.request
import planet

In [9]:
## variables to change
PLANET_API_KEY = 'ba80d8f478df4ec6ba4247ca3c9c8112' # <= insert API key here 
month_yr = "2021-01" ## year and month for basemap
DIR = '/Users/michaelcecil/Rprojects/geog287387_s/materials/data/Zambia/NICFI' ## (change to your output path)

In [10]:
## enter your API key below


# https://developers.planet.com/planetschool/downloading-quads-from-a-mosaic-using-basemaps-api/
#setup API KEY
PLANET_API_KEY = 'ba80d8f478df4ec6ba4247ca3c9c8112' # <= insert API key here 

#setup Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"
#setup session
session = requests.Session()
#authenticate
session.auth = (PLANET_API_KEY, "") 

In [11]:


mosaic_name = "planet_medres_normalized_analytic_" + month_yr + "_mosaic"

parameters = {
    "name__is" : mosaic_name # <= customize to your use case
}
#make get request to access mosaic from basemaps API
res = session.get(API_URL, params = parameters)
print("status code:", res.status_code) #response status code


#print metadata for mosaic
mosaic = res.json()
print("mosaic metadata:", json.dumps(mosaic, indent=2))

status code: 200
mosaic metadata: {
  "_links": {
    "_self": "https://api.planet.com/basemaps/v1/mosaics?api_key=ba80d8f478df4ec6ba4247ca3c9c8112"
  },
  "mosaics": [
    {
      "_links": {
        "_self": "https://api.planet.com/basemaps/v1/mosaics/08edb2bf-3f36-4ad5-9330-182e5245a738?api_key=ba80d8f478df4ec6ba4247ca3c9c8112",
        "quads": "https://api.planet.com/basemaps/v1/mosaics/08edb2bf-3f36-4ad5-9330-182e5245a738/quads?api_key=ba80d8f478df4ec6ba4247ca3c9c8112&bbox={lx},{ly},{ux},{uy}",
        "tiles": "https://tiles.planet.com/basemaps/v1/planet-tiles/planet_medres_normalized_analytic_2021-01_mosaic/gmap/{z}/{x}/{y}.png?api_key=ba80d8f478df4ec6ba4247ca3c9c8112"
      },
      "bbox": [
        -179.999999974944,
        -30.145127179527,
        179.999999975056,
        30.145127179625
      ],
      "coordinate_system": "EPSG:3857",
      "datatype": "uint16",
      "first_acquired": "2021-01-01T00:00:00.000Z",
      "grid": {
        "quad_size": 4096,
        "resol

In [12]:

mosaic_id = mosaic['mosaics'][0]['id'] #get mosaic id
print('Mosaic id: '+ mosaic_id)

#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']
print("mosaic bbox: ", mosaic_bbox)
mosaic_bbox


#convert mosaic bbox to string for search params
#string_bbox = ','.join(map(str, mosaic_bbox))
#print(string_bbox)

# set your own bounding box
# use format below (lowerx,lowery,upperx,uppery)
string_bbox = "28.25102349,-15.55190159,28.25102349,-15.55190159"
print("string bbox:", string_bbox)


Mosaic id: 08edb2bf-3f36-4ad5-9330-182e5245a738
mosaic bbox:  [-179.999999974944, -30.145127179527, 179.999999975056, 30.145127179625]
string bbox: 28.25102349,-15.55190159,28.25102349,-15.55190159


In [13]:
#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}

#accessing quads using metadata from mosaic
quads_url = "{}/{}/quads".format(API_URL, mosaic_id)
res = session.get(quads_url, params=search_parameters, stream=True)
quads = res.json()
items = quads['items']
print(items)

[{'_links': {'download': 'https://api.planet.com/basemaps/v1/mosaics/08edb2bf-3f36-4ad5-9330-182e5245a738/quads/1184-934/full?api_key=ba80d8f478df4ec6ba4247ca3c9c8112'}, 'bbox': [28.125000000000004, -15.623036831528252, 28.300781250000004, -15.453680224345847], 'id': '1184-934', 'percent_covered': None}]


In [15]:


#iterate over quad download links and save to folder by id
for i in items:
    link = i['_links']['download']
    name = i['id'] + "_" + month_yr
    name = name + '.tiff'
    filename = os.path.join(DIR, name)

    #checks if file already exists before saving
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(link, filename)